# Multiple Results

In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.

In [ ]:
!python -m pip install semantic-kernel==0.4.4.dev0

In [9]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.ai_request_settings import AIRequestSettings
from semantic_kernel.connectors.ai.open_ai.request_settings.open_ai_request_settings import (
    OpenAITextRequestSettings,
)
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureChatCompletion, OpenAITextCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

First, we will set up the text and chat services we will be submitting prompts to.

In [16]:
kernel = sk.Kernel()

# Configure Azure LLM service
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
azure_text_service = AzureTextCompletion(deployment_name="text-davinci-002", endpoint=endpoint, api_key=api_key)    # set the deployment name to the value of your text model
azure_chat_service = AzureChatCompletion(deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key)   # set the deployment name to the value of your chat model


# Configure OpenAI service
api_key, org_id = sk.openai_settings_from_dot_env()
oai_text_service = OpenAITextCompletion(ai_model_id="text-davinci-003", api_key=api_key, org_id=org_id)
oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)

# Configure Hugging Face service
hf_text_service = HuggingFaceTextCompletion(ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.

In [11]:
request_settings = OpenAITextRequestSettings(extension_data = { "max_tokens": 80, "temperature": 0.7, "top_p": 1, "frequency_penalty": 0.5, "presence_penalty": 0.5, "number_of_responses": 3})

## Multiple Open AI Text Completions

In [12]:
prompt = "what is the purpose of a rubber duck?"
results = await oai_text_service.complete_async(prompt=prompt, settings=request_settings)
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

Result 1: 

The purpose of a rubber duck is primarily for use as a bath toy for children. It can also be used to teach young children basic water safety and swimming skills.
Result 2: 

The purpose of a rubber duck is to provide a fun bathtime companion for children and give them something to focus on while they are learning how to bathe. Many people also use rubber ducks as decorative items in the home or as a gift.
Result 3: 

A rubber duck is often used as a toy, but it can also serve several other purposes. Some people use them to help children learn about water safety, or as a calming tool for young children to help them relax during bath time. They can provide a fun and safe way for children to play in the water, and they are also popular bathroom decorations.


## Multiple Azure Open AI Text Completions

In [17]:
prompt = "provide me a list of possible meanings for the acronym 'ORLD'"
results = await azure_text_service.complete_async(prompt=prompt, settings=request_settings)
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

AIException: (<ErrorCodes.ServiceError: 6>, "<class 'semantic_kernel.connectors.ai.open_ai.services.azure_text_completion.AzureTextCompletion'> service failed to complete the prompt", AuthenticationError("Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}"))

## Multiple Hugging Face Text Completions

In [20]:
from semantic_kernel.connectors.ai.hugging_face.hf_request_settings import (
    HuggingFaceRequestSettings,
)

request_settings = HuggingFaceRequestSettings(extension_data = { "max_new_tokens": 80, "temperature": 0.7, "top_p": 1})

In [24]:
prompt = "The purpose of a rubber duck is"
results = await hf_text_service.complete_async(prompt=prompt, request_settings=request_settings)
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

AIException: ('Hugging Face completion failed', TypeError('HuggingFaceRequestSettings.prepare_settings_dict() takes 1 positional argument but 2 were given'), None)

Here, we're setting up the settings for Chat completions.

In [ ]:
chat_request_settings = ChatRequestSettings(
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3
)

## Multiple OpenAI Chat Completions

In [ ]:
role = "user"
content = "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
message = { "role":role, "content":content }
results = await oai_chat_service.complete_chat_async(messages=[message], settings=chat_request_settings)
i = 0
for result in results:
    print(f"Result {i}: {result}")
    i += 1

## Multiple Azure OpenAI Chat Completions

In [ ]:
role = "user"
content = "Tomorow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
message = { "role":role, "content":content }
results = await azure_chat_service.complete_chat_async(messages=[message], settings=chat_request_settings)
i = 0
for result in results:
    print(f"Result {i}: {result}")
    i += 1

## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.

In [ ]:
import os
from IPython.display import clear_output

if os.name == "nt":
    clear = "cls"
else:
    clear = "clear"

prompt = "what is the purpose of a rubber duck?"
stream = oai_text_service.complete_stream_async(prompt=prompt, settings=request_settings)
texts = [""] * request_settings.number_of_responses
async for results in stream:
    i = 1
    clear_output(wait=True)
    for result in results:
        texts[i - 1] += result
        print(f"Result {i}: {texts[i - 1]}")
        i += 1

print("----------------------------------------")
